In [1]:
import chess
import chess.engine
import random
import numpy
import numpy as np
import os


# this function will create our x (board)
def random_board(max_depth=200):
    board = chess.Board()
    depth = random.randrange(0, max_depth)

    for _ in range(depth):
        all_moves = list(board.legal_moves)
        random_move = random.choice(all_moves)
        board.push(random_move)
        if board.is_game_over():
            break

    return board


# this function will create our f(x) (score)
def stockfish(board, depth):
    with chess.engine.SimpleEngine.popen_uci('stockfish') as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        score = result['score'].white().score()
        return score

In [4]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
    # this is the 3d matrix
    board3d = numpy.zeros((8, 8, 8), dtype=numpy.int8)

    # here we add the pieces's view on the matrix
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = numpy.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = numpy.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = -1

  # add attacks and valid moves too
  # so the network knows what is being attacked
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[6][i][j] = 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[7][i][j] = 1
    board.turn = aux
    return board3d

In [5]:
split_dims(chess.Board())

array([[[ 0,  0,  0,  0,  0,  0,  0,  0],
        [-1, -1, -1, -1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0]],

       [[ 0, -1,  0,  0,  0,  0, -1,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  0,  0,  0,  0,  1,  0]],

       [[ 0,  0, -1,  0,  0, -1,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  

In [3]:
def make_dir_safe(dir_path):
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)

In [4]:
make_dir_safe('white_random_games')
make_dir_safe('black_random_games')


black_game_data = []
white_game_data = []

In [5]:
num_moves = 0
max_moves = num_moves + 100000

In [6]:
while num_moves < max_moves:
    num_moves += 1
    board = random_board()
    score = stockfish(board,10)
    
    if board.turn:
        path = os.path.join('white_random_games',f'{num_moves}')
        np.save(path,split_dims(board))
        white_game_data.append({
            'game_name':'random',
            'move':num_moves,
            'path':path,
            'score':score,
        })
    else:
        path = os.path.join('black_random_games',f'{num_moves}')
        np.save(path,split_dims(board))
        black_game_data.append({
            'game_name':'random',
            'move':num_moves,
            'path':path,
            'score':score,
        })
    if num_moves%100 == 0:
        print(f'{num_moves} moves done')

100 moves done
200 moves done
300 moves done
400 moves done
500 moves done
600 moves done
700 moves done
800 moves done
900 moves done
1000 moves done
1100 moves done
1200 moves done
1300 moves done
1400 moves done
1500 moves done
1600 moves done
1700 moves done
1800 moves done
1900 moves done
2000 moves done
2100 moves done
2200 moves done
2300 moves done
2400 moves done
2500 moves done
2600 moves done
2700 moves done
2800 moves done
2900 moves done
3000 moves done
3100 moves done
3200 moves done
3300 moves done
3400 moves done
3500 moves done
3600 moves done
3700 moves done
3800 moves done
3900 moves done
4000 moves done
4100 moves done
4200 moves done
4300 moves done
4400 moves done
4500 moves done
4600 moves done
4700 moves done
4800 moves done
4900 moves done
5000 moves done
5100 moves done
5200 moves done
5300 moves done
5400 moves done
5500 moves done
5600 moves done
5700 moves done
5800 moves done
5900 moves done
6000 moves done
6100 moves done
6200 moves done
6300 moves done
6

49000 moves done
49100 moves done
49200 moves done
49300 moves done
49400 moves done
49500 moves done
49600 moves done
49700 moves done
49800 moves done
49900 moves done
50000 moves done
50100 moves done
50200 moves done
50300 moves done
50400 moves done
50500 moves done
50600 moves done
50700 moves done
50800 moves done
50900 moves done
51000 moves done
51100 moves done
51200 moves done
51300 moves done
51400 moves done
51500 moves done
51600 moves done
51700 moves done
51800 moves done
51900 moves done
52000 moves done
52100 moves done
52200 moves done
52300 moves done
52400 moves done
52500 moves done
52600 moves done
52700 moves done
52800 moves done
52900 moves done
53000 moves done
53100 moves done
53200 moves done
53300 moves done
53400 moves done
53500 moves done
53600 moves done
53700 moves done
53800 moves done
53900 moves done
54000 moves done
54100 moves done
54200 moves done
54300 moves done
54400 moves done
54500 moves done
54600 moves done
54700 moves done
54800 moves do

97200 moves done
97300 moves done
97400 moves done
97500 moves done
97600 moves done
97700 moves done
97800 moves done
97900 moves done
98000 moves done
98100 moves done
98200 moves done
98300 moves done
98400 moves done
98500 moves done
98600 moves done
98700 moves done
98800 moves done
98900 moves done
99000 moves done
99100 moves done
99200 moves done
99300 moves done
99400 moves done
99500 moves done
99600 moves done
99700 moves done
99800 moves done
99900 moves done
100000 moves done


In [7]:
import pandas as pd

black_df = pd.DataFrame(black_game_data)
white_df = pd.DataFrame(white_game_data)

black_df['path'] += '.npy'
white_df['path'] += '.npy'

In [8]:
black_df.to_csv('black_random_1.csv',index=False)
white_df.to_csv('white_random_1.csv',index=False)